In [130]:
# Задание на повторение материала предыдущего семестра

In [131]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [132]:
# Генерируем уникальный seed
my_code = "rOMANENKOV"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [133]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [134]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [135]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [136]:
example_data.shape

(2315, 7)

In [137]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [138]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [139]:
train.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
1896,2012,Kemerovo Oblast,-1.4,13.8,15.2,66.22,85.40
2242,2016,Kurgan Oblast,-3.5,12.4,15.9,88.18,61.80
1325,2005,Perm Krai,-7.0,10.9,18.1,58.79,75.30
129,1991,Novgorod Oblast,-3.7,10.9,14.6,83.71,69.89
1760,2010,Saratov Oblast,-4.9,10.8,15.7,64.20,74.40


In [140]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [141]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [142]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [143]:
sc_val

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.703704,0.341523,0.242424,0.497696,0.325452,0.684431,Primorsky Krai
1,0.037037,0.643735,0.376623,0.064516,0.371871,0.893937,Khanty–Mansi Autonomous Okrug – Yugra
2,0.518519,0.34398,0.229437,0.474654,0.077886,0.90965,Magadan Oblast
3,0.740741,0.233415,0.207792,0.654378,0.501043,0.546943,Oryol Oblast
4,0.62963,0.385749,0.329004,0.511521,0.343707,0.730261,Irkutsk Oblast
...,...,...,...,...,...,...,...
458,0.037037,0.44226,0.329004,0.391705,0.769819,0.3978,Stavropol Krai
459,0.62963,0.277641,0.194805,0.548387,0.365786,0.739426,Samara Oblast
460,0.481481,0.144963,0.125541,0.732719,0.577538,0.587927,Bryansk Oblast
461,0.37037,0.081081,0.056277,0.778802,0.659771,0.60089,Novgorod Oblast


In [144]:
# Вспоминаем алгоритмы решения задачи регрессии
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'mse', 'friedman_mse', 'mae', 'poisson'
r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [145]:
r_models

[LinearRegression(),
 Lasso(),
 Ridge(),
 ElasticNet(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [146]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [147]:
x_labels

['year', 'npg', 'birth_rate', 'death_rate', 'gdw']

In [148]:
y_labels

['urbanization']

In [149]:
# Обучаем модели
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [150]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

LinearRegression() 	 0.018334784325700747
Lasso() 	 0.028527646902099073
Ridge() 	 0.01832324049051655
ElasticNet() 	 0.028527646902099073
KNeighborsRegressor() 	 0.009792131592070472
KNeighborsRegressor(n_neighbors=10) 	 0.010700597078351144
KNeighborsRegressor(n_neighbors=15) 	 0.011750117503498105
SVR(kernel='linear') 	 0.018376805479515364
SVR(kernel='poly') 	 0.013056519391521379
SVR() 	 0.011537874597555538
SVR(kernel='sigmoid') 	 25.90830470287517
DecisionTreeRegressor() 	 0.022379730633334424
DecisionTreeRegressor(criterion='friedman_mse') 	 0.022329938427441985
DecisionTreeRegressor(criterion='absolute_error') 	 0.01715521389197987


In [151]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [152]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.013564628622507093


In [153]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l1', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='linear'))
c_models.append(SVC(kernel='poly'))
c_models.append(SVC(kernel='rbf'))
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='random'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='random'))

In [154]:
c_models

[LogisticRegression(max_iter=500, penalty='none', solver='saga'),
 LogisticRegression(max_iter=500, penalty='l1', solver='saga'),
 LogisticRegression(solver='saga'),
 LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 LogisticRegression(),
 MultinomialNB(alpha=0.0),
 MultinomialNB(alpha=0.5),
 MultinomialNB(),
 KNeighborsClassifier(),
 KNeighborsClassifier(n_neighbors=10),
 KNeighborsClassifier(n_neighbors=15),
 SVC(kernel='linear'),
 SVC(kernel='poly'),
 SVC(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random'),
 DecisionTreeClassifier(criterion='entropy'),
 DecisionTreeClassifier(criterion='entropy', splitter='random')]

In [155]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [156]:
x_train

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.814815,0.356265,0.329004,0.552995,0.499305,0.808825
1,0.962963,0.304668,0.268398,0.585253,0.881085,0.499804
2,0.555556,0.218673,0.203463,0.686636,0.370132,0.676575
3,0.037037,0.299754,0.203463,0.525346,0.803373,0.605735
4,0.740741,0.27027,0.199134,0.576037,0.464186,0.66479
...,...,...,...,...,...,...
1384,0.444444,0.063882,0.121212,0.880184,0.600661,0.60364
1385,0.259259,0.154791,0.060606,0.645161,0.820584,0.437737
1386,0.555556,0.125307,0.121212,0.764977,0.493915,0.583606
1387,0.333333,0.304668,0.160173,0.470046,0.4508,0.558858


In [157]:
# Обучаем модели
for model in c_models:
    print(model)
    model.fit(x_train, y_train)

LogisticRegression(max_iter=500, penalty='none', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(max_iter=500, penalty='l1', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(solver='saga')
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')
LogisticRegression()
MultinomialNB(alpha=0.0)
MultinomialNB(alpha=0.5)
MultinomialNB()
KNeighborsClassifier()
KNeighborsClassifier(n_neighbors=10)
KNeighborsClassifier(n_neighbors=15)
SVC(kernel='linear')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')
DecisionTreeClassifier(criterion='entropy')
DecisionTreeClassifier(criterion='entropy', splitter='random')


In [158]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

LogisticRegression(max_iter=500, penalty='none', solver='saga') 	 0.3694743779528699
LogisticRegression(max_iter=500, penalty='l1', solver='saga') 	 0.12942710411265868
LogisticRegression(solver='saga') 	 0.05744133398958935
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga') 	 0.05762912566971263
LogisticRegression() 	 0.05729734550866783
MultinomialNB(alpha=0.0) 	 0.04137554737681616
MultinomialNB(alpha=0.5) 	 0.03854126462414953
MultinomialNB() 	 0.03689504996297981
KNeighborsClassifier() 	 0.3553474292590918
KNeighborsClassifier(n_neighbors=10) 	 0.26203844708682583
KNeighborsClassifier(n_neighbors=15) 	 0.22047061702475648
SVC(kernel='linear') 	 0.08073990369898658
SVC(kernel='poly') 	 0.6802202002119754
SVC() 	 0.20855739087202696
SVC(kernel='sigmoid') 	 3.715831765716807e-05
DecisionTreeClassifier() 	 0.6277777358787595
DecisionTreeClassifier(splitter='random') 	 0.591852450995441
DecisionTreeClassifier(criterion='entropy') 	 0.6354577412733937
DecisionTreeClas

In [159]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

SVC(kernel='poly')

In [160]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.6588807081742506


In [161]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [162]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.814815,0.356265,0.329004,0.552995,0.499305,0.808825
1,0.962963,0.304668,0.268398,0.585253,0.881085,0.499804
2,0.555556,0.218673,0.203463,0.686636,0.370132,0.676575
3,0.037037,0.299754,0.203463,0.525346,0.803373,0.605735
4,0.740741,0.27027,0.199134,0.576037,0.464186,0.66479
...,...,...,...,...,...,...
458,0.37037,0.299754,0.142857,0.460829,0.396384,0.568679
459,0.185185,0.167076,0.082251,0.645161,0.769124,0.596962
460,0.925926,0.535627,0.480519,0.37788,0.607615,0.461831
461,0.592593,0.383292,0.207792,0.396313,0.138734,0.723714


In [163]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [164]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.295027717026857
0.2453793801929358
0.2357203964229925
0.23073478758161217


In [165]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=5)
0.295027717026857


In [166]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25),
    SVR(kernel='linear'),
    SVR(kernel='poly'),
    SVR(kernel='rbf'),
    SVR(kernel='sigmoid'),
    DecisionTreeRegressor(criterion='squared_error'),
    DecisionTreeRegressor(criterion='friedman_mse'),
    DecisionTreeRegressor(criterion='absolute_error')
]

In [167]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[ElasticNet(alpha=0.5), LinearRegression(), KNeighborsRegressor(n_neighbors=15), KNeighborsRegressor()]


In [168]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv", decimal=',') 
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,-3.0,-0.3,-0.4
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,-2.0,0.6,-1.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,-7.0,-1.8,-0.3
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,-1.0,1.2,-1.3
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,-2.0,2.5,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,-1.0,5.9,1.5
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,-1.0,5.2,3.1
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,1.0,1.4,1.9
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,-4.0,2.7,1.7


In [169]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['wl_change', 'humidity', 'pressure', 'wind', 'weather']


In [170]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

data = data.drop(['precipitation', 'temperature', 'visibility', 'fire', 'temp_change', 'pressure_change'],axis=1)


In [171]:
data

,water_level,humidity,wind,weather,pressure,wl_change
0,468.0,31.0,4.0,0.0,993.8,-3.0
1,140.0,22.0,1.0,0.0,1004.1,-2.0
2,464.0,28.0,2.0,1.0,1001.2,-7.0
3,467.0,26.0,3.0,0.0,992.3,-1.0
4,138.0,22.0,1.0,0.0,1001.3,-2.0
...,...,...,...,...,...,...
1434,474.0,95.0,1.0,3.0,999.9,-1.0
1435,65.0,93.0,4.0,3.0,983.9,-1.0
1436,159.0,83.0,2.0,1.0,969.7,1.0
1437,99.0,91.0,3.0,3.0,995.4,-4.0


In [172]:
data.shape
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [173]:
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [174]:
train.head()

,water_level,humidity,wind,weather,pressure,wl_change
431,578.0,65.0,1.0,2.0,997.3,-3.0
1244,239.0,60.0,3.0,1.0,997.8,-7.0
811,83.0,68.0,1.0,2.0,927.3,-7.0
885,328.0,59.0,6.0,4.0,986.6,13.0
225,344.0,96.0,1.0,5.0,997.7,-1.0


In [175]:
columns_to_scale = ['wl_change', 'pressure', 'weather', 'wind', 'humidity']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['wl_change', 'pressure', 'weather', 'wind',
                                  'humidity'])])

In [176]:
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))
column_names = columns_to_scale + ['water_level']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names
sc_val

,wl_change,pressure,weather,wind,humidity,water_level
0,0.393204,0.857463,0.125,0.2,0.921348,115.0
1,0.388350,0.811194,0.125,0.1,0.573034,31.0
2,0.393204,0.872388,0.750,0.1,1.000000,227.0
3,0.383495,0.771642,0.000,0.3,0.584270,312.0
4,0.436893,0.724627,0.000,0.1,0.191011,498.0
...,...,...,...,...,...,...
283,0.388350,0.750746,0.125,0.3,0.516854,225.0
284,0.393204,0.905970,0.250,0.2,0.415730,99.0
285,0.621359,0.773881,0.125,0.2,0.707865,429.0
286,0.383495,0.618657,0.125,0.4,0.539326,638.0


In [177]:
r_models = []

r_models.append(ElasticNet(alpha=0.5))
r_models.append(ElasticNet(alpha=0.5, l1_ratio=0.75))

r_models.append(SVR())

r_models.append(Ridge())
r_models

[ElasticNet(alpha=0.5), ElasticNet(alpha=0.5, l1_ratio=0.75), SVR(), Ridge()]

In [178]:
x_labels = column_names[0:-1]
y_labels = ['water_level']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [179]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
SVR()
Ridge()


In [180]:
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 41574.04517406472
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 41077.910579626514
SVR() 	 46517.52494616646
Ridge() 	 40255.303144248755


In [181]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

Ridge()

In [182]:
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

48928.88902222304


In [218]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [219]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[KNeighborsClassifier(n_neighbors=10), KNeighborsClassifier(n_neighbors=25), LogisticRegression(l1_ratio=0.75, penalty='elasticnet', solver='saga'), DecisionTreeClassifier(criterion='entropy'), SVC(kernel='sigmoid')]


In [220]:
# Загрузим данные для задачи классификации
newdata = pd.read_csv("datasets/zoo2.csv")
newdata

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [221]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(newdata.columns)
n_x = 8

y_newlabel = 'class_type'
x_newlabels = random.sample(columns[1:-1], n_x)

print(x_newlabels)

['tail', 'hair', 'aquatic', 'backbone', 'legs', 'breathes', 'toothed', 'fins']


In [222]:
labels = x_newlabels.copy()
labels.append(y_newlabel)
temp_data = newdata[labels]
sec_data = temp_data

In [223]:
sec_data

,tail,hair,aquatic,backbone,legs,breathes,toothed,fins,class_type
0,1,0,1,1,4,1,0,0,3
1,1,0,0,1,4,1,1,0,3
2,1,0,0,1,4,1,1,0,3
3,1,0,0,1,4,1,1,0,3
4,1,0,0,1,4,1,1,0,3
5,1,0,0,1,0,1,1,0,3
6,1,0,0,1,0,1,1,0,3
7,1,0,0,1,0,1,1,0,3
8,1,0,1,1,4,1,1,0,3
9,1,0,1,1,4,1,1,0,3


In [224]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

sec_data.shape
val_test_size2 = round(0.2*len(sec_data))
print(val_test_size2)

9


In [225]:
random_state = my_seed
train_val2, test2 = train_test_split(sec_data, test_size=val_test_size2, random_state=random_state)
sec_train, val2 = train_test_split(train_val2, test_size=val_test_size2, random_state=random_state)
print(len(train2), len(val2), len(test2))

NameError: name 'train2' is not defined

In [226]:
sec_train.head()

,tail,hair,aquatic,backbone,legs,breathes,toothed,fins,class_type
13,1,0,1,1,0,0,0,1,4
29,0,1,0,0,6,1,0,0,6
26,0,0,0,0,6,1,0,0,6
9,1,0,1,1,4,1,1,0,3
17,1,0,1,1,0,0,0,1,4


In [227]:
columns_to_scale2 = x_newlabels

ct2 = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale2)], remainder='passthrough')
ct2.fit(sec_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['tail', 'hair', 'aquatic', 'backbone', 'legs',
                                  'breathes', 'toothed', 'fins'])])

In [228]:
sc_train2 = pd.DataFrame(ct2.transform(sec_train))
sc_test2 = pd.DataFrame(ct2.transform(test2))
sc_val2 = pd.DataFrame(ct2.transform(val2))
column_names2 = columns_to_scale2 + ['class_type']
sc_train2.columns = column_names2
sc_test2.columns = column_names2
sc_val2.columns = column_names2
sc_val2

,tail,hair,aquatic,backbone,legs,breathes,toothed,fins,class_type
0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.0,7.0
1,1.0,0.0,1.0,1.0,0.00,0.0,1.0,1.0,4.0
2,1.0,0.0,0.0,1.0,0.00,1.0,1.0,0.0,3.0
3,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0,6.0
4,1.0,0.0,1.0,1.0,0.00,0.0,0.0,1.0,4.0
5,0.0,0.0,1.0,0.0,0.00,0.0,0.0,0.0,7.0
6,1.0,0.0,1.0,1.0,0.50,1.0,0.0,0.0,3.0
7,0.0,0.0,1.0,0.0,0.00,0.0,0.0,0.0,7.0
8,1.0,0.0,0.0,1.0,0.50,1.0,1.0,0.0,3.0


In [229]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='l1', solver='saga'))


# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier())
c_models.append(DecisionTreeClassifier(splitter='random'))
c_models
#[MultinomialNB(), KNeighborsClassifier, DecisionTreeClassifier(), LogisticRegression, KNeighborsClassifier]

[LogisticRegression(penalty='l1', solver='saga'),
 KNeighborsClassifier(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random')]

In [230]:
x_labels2 = column_names2[0:-1]
y_labels2 = ['class_type']

x_train2 = sc_train2[x_labels2]
x_test2 = sc_test2[x_labels2]
x_val2 = sc_val2[x_labels2]

y_train2 = np.ravel(sc_train2[y_labels2])
y_test2 = np.ravel(sc_test2[y_labels2])
y_val2 = np.ravel(sc_val2[y_labels2])
x_train2

,tail,hair,aquatic,backbone,legs,breathes,toothed,fins
0,1.0,0.0,1.0,1.0,0.00,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.75,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0
3,1.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
4,1.0,0.0,1.0,1.0,0.00,0.0,0.0,1.0
5,0.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0
7,0.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
8,1.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0


In [231]:
# Обучаем модели
for model2 in c_models:
    print(model2)
    model2.fit(x_train2, y_train2)

LogisticRegression(penalty='l1', solver='saga')
KNeighborsClassifier()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [232]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model2 in c_models:
    val_pred2 = model2.predict(x_val2)
    f1 = f1_score(y_val2, val_pred2, average='weighted')
    f1s.append(f1)
    print(model2, '\t', f1)

LogisticRegression(penalty='l1', solver='saga') 	 0.6000000000000001
KNeighborsClassifier() 	 0.6000000000000001
SVC(kernel='sigmoid') 	 0.324074074074074
DecisionTreeClassifier() 	 0.7777777777777778
DecisionTreeClassifier(splitter='random') 	 0.7777777777777778


In [233]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

DecisionTreeClassifier()

In [234]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred2 = best_c_model.predict(x_test2)
f1 = f1_score(y_test2, test_pred2, average='weighted')
print(f1)

0.8395061728395061


In [235]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=40))

In [236]:
# Выделим используемые параметры
x_labels2 = column_names2[0:-1]
x_data2 = pd.concat([sc_train2[x_labels2], sc_val2[x_labels2], sc_test2[x_labels2]])
x_data2

,tail,hair,aquatic,backbone,legs,breathes,toothed,fins
0,1.0,0.0,1.0,1.0,0.00,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.75,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0
3,1.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
4,1.0,0.0,1.0,1.0,0.00,0.0,0.0,1.0
5,0.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0
7,0.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
8,1.0,0.0,1.0,1.0,0.50,1.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.75,1.0,0.0,0.0


In [237]:
# Произведем кластеризацию
for model2 in k_models:
    model2.fit(x_data2)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (13) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (13) found smaller than n_clusters (40). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until


In [240]:
# Оценим качество результата
sils = []
for model2 in k_models:
    cluster_labels = model2.predict(x_data2)
    s = silhouette_score(x_data2, cluster_labels)
    sils.append(s)
    print(s)

0.7047248792791888
0.8050921844164834
0.8837209302325582
0.8837209302325582


In [241]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=20)
0.8837209302325582
